In [1]:
import pandas as pd
import re

In [2]:
client_info_df = pd.read_csv(r'E:\wd_work\unti_cheat\D1_B_0201_0613_0615_B.csv')

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#######城市变更次数########################

In [6]:
client_info_df.ACTION_CONTROL.value_counts()

my_home_address        815662
my_company_name        524858
my_contact_phone       376697
my_contact             319647
my_work_address        137682
city                    30662
my_phone                12943
my_contact_relation      9442
Name: ACTION_CONTROL, dtype: int64

In [8]:
client_city_df = client_info_df[client_info_df.ACTION_CONTROL=='city']

In [11]:
client_city_df.columns

Index(['ACTION_CONTROL', 'ACTION_GROUP', 'ACTION_PAGE', 'ACTION_TEXT',
       'ACTION_VALUE', 'CLIENT_TIME', 'CREATE_DATE', '中文匹配', 'D3', 'D10', 'F6',
       'GETTEL(A.MOBILE)', 'ID', 'MACHINE_UUID', 'MAX_MAXOVERDUEDAYS', '记录数',
       'PUTOUTDATE_TIMES', 'T3%', 'T30#', 'T3#', 'T10#', '[T3->T10]%', 'T10% ',
       'T20#', 'TEL', 'USER_ID', '日(APPLY_DATE)', '日(MIN_PUTOUTDATE)'],
      dtype='object')

In [15]:
city_changed_cnt = client_city_df.drop_duplicates(['USER_ID','ACTION_VALUE'])[['USER_ID','ACTION_VALUE']].groupby('USER_ID').count().reset_index()

In [514]:
#########详细地址变更次数########################

In [ ]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
client_addr_df.ACTION_VALUE = client_addr_df.ACTION_VALUE.apply(lambda x:re.sub(u"[a-z？]",'',str(x)))
client_addr = []
user_idlist = []
client_addr_df['str_len'] = client_addr_df.ACTION_VALUE.str.len()
client_addr_df['str_len1'] =client_addr_df['str_len'].shift(-1).fillna(list(client_addr_df['str_len'])[-1])
client_addr_df['str_len_dif'] = client_addr_df['str_len1']-client_addr_df['str_len']
client_addr_df['str_len_dif1'] =client_addr_df['str_len_dif'].shift(1)
######################################################################################################################################
for user_id in set(list(client_addr_df.USER_ID)):
#     print(user_id)
    aa = client_addr_df[client_addr_df.USER_ID==user_id]
    aaaa = aa.iloc[1:-1,:]
    aaa = aaaa[(aaaa.str_len_dif1>=0) & (aaaa.str_len_dif<0)]
    if len(aaa)>=1:
        bb = list(aaa.ACTION_VALUE)
        bb.append(list(aa.iloc[-1:,:].ACTION_VALUE)[-1])
        client_addr.append(bb)
    else:
        client_addr.append(list(aa.iloc[-1:,:].ACTION_VALUE)[-1])
    user_idlist.append(user_id)
#######################################################################################################################################    
client_addr1 = []
for addr in client_addr:
    aaaaa = []
    if type(addr) != str:       
        for addr1 in addr:
#             print(len(addr1))
            if len(addr1)>5:
                aaaaa.append(addr1)
        client_addr1.append(aaaaa)
    else:
        if len(addr)>5:
            client_addr1.append(addr)
        else:
            client_addr1.append(' ')
######################################################################################################################################
id_list = []
c = 0
for addr in client_addr1:
    
    if type(addr) != str and len(addr)>1:       
        id_list.extend([user_idlist[c],]*len(addr))
        
    elif type(addr) == str or len(addr)==1:
        id_list.append(user_idlist[c])
    c += 1
addr_list = []
for addr in client_addr1:
    if type(addr) != str:       
        addr_list.extend(addr)
        
    elif type(addr) == str :
        addr_list.append(addr)
#################################################################################################################################
ADDR_DF = pd.DataFrame({'USER_ID':id_list,'ADDR':addr_list})
ADDR_DF['ADDR_LEN'] = ADDR_DF.ADDR.str.len()
ADDR_DF = ADDR_DF.sort_values(by=['USER_ID','ADDR_LEN'],ascending=False)
addr_cnt_df = ADDR_DF.groupby('USER_ID').count().reset_index()[['USER_ID','ADDR']].rename(columns={'ADDR':'addr_cnt'})
ADDR_DF1 = pd.merge(ADDR_DF,addr_cnt_df,on='USER_ID',how='left')
ADDR_DF2 = ADDR_DF1[ADDR_DF1.addr_cnt>1]
import difflib
data_box = []
for user_id in set(list(ADDR_DF2.USER_ID)):
    ADDR_DF3 = ADDR_DF2[ADDR_DF2.USER_ID==user_id]
    addr1 = list(ADDR_DF3.ADDR)[0]
    for i in range(1,len(ADDR_DF3)):
        if difflib.SequenceMatcher(None,str(addr1),str( list(ADDR_DF3.ADDR)[i])).ratio()<0.4:
            data_box.append((user_id,addr1,list(ADDR_DF3.ADDR)[i]))
ADDR_DF3 = pd.DataFrame(data_box,columns=['USER_ID','ADDR1','ADDR2'])
ADDR_DF3['ADDR_LEN'] = ADDR_DF3.ADDR2.str.len()
ADDR_DF3 = ADDR_DF3.sort_values(by='ADDR_LEN')
import copy
dt = []
for user_id in set(list(ADDR_DF3.USER_ID)):
    addr_df = ADDR_DF3[ADDR_DF3.USER_ID==user_id]
    addr_df = addr_df.sort_values(by='ADDR_LEN',ascending=False)
    aa = list(addr_df.ADDR2)
    aaa = copy.deepcopy(aa)
    print(aaa)
    if addr_df.shape[0]>1:
        for i in range(len(aa)-1):
            for j in range(i+1,len(aa)):
                if difflib.SequenceMatcher(None,str(aa[i]),str(aa[j])).ratio()>0.3:
#                     print(aa[j])
                    if aa[j] in aaa:
                        aaa.remove(aa[j])
    
    dt.append((user_id,aaa))
ADDR_DF4 = pd.DataFrame(dt,columns=['USER_ID','ADDR3'])    
ADDR_DF5 = pd.merge(ADDR_DF3,ADDR_DF4,on='USER_ID',how='left').drop_duplicates(['USER_ID','ADDR1'])
addr_lists = []
for i in range(len(ADDR_DF5)):
    addr_list  = list(ADDR_DF5.ADDR3)[i]
    addr_list.append(list(ADDR_DF5.ADDR1)[i])
    addr_lists.append([len(addr_list),addr_list])
ADDR_DF5['addr_changed_cnt'] = [addr[0] for addr in addr_lists]
ADDR_DF5['addr_changed'] = [addr[1] for addr in addr_lists]
ADDR_DF6 = ADDR_DF5[['USER_ID','addr_changed_cnt','addr_changed']]
ADDR_DF7 = ADDR_DF1[ADDR_DF1.addr_cnt==1][['USER_ID','addr_cnt','ADDR']]
ADDR_DF7.columns=ADDR_DF6.columns
ADDR_DF8 = pd.concat([ADDR_DF7,ADDR_DF6],ignore_index=True)
ADDR_DF8.addr_changed_cnt.value_counts()

In [517]:
#####################是否精确到详细地址####################################

In [518]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address'].sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='last')
bool_list = []
for addr in list(client_addr_df.ACTION_VALUE):
    c = 0
    for i in range(10):
        if str(i) in addr:
            c+=1
    if c>0:
        bool_list.append(1)
    else:
        bool_list.append(0)
client_addr_df['is_addr_accurate'] = bool_list
is_addr_accurate_df = client_addr_df[['USER_ID','is_addr_accurate']]

Wall time: 1.38 s


In [520]:
######################模糊详细地址##############################

In [521]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
client_addr_df1 = client_addr_df[['USER_ID','ID','ACTION_VALUE']]
client_addr_df1['addr_len'] = client_addr_df1.ACTION_VALUE.str.len()
def stat_addr(df):
    
    df['addr_len1'] = df['addr_len'].shift(-1).fillna(999)
    df['dif'] = df['addr_len1']-df['addr_len']
    df1 = df[df.dif<0]
    df1.index = range(len(df1))
    df1['ID1'] = df1['ID'].shift(-1)
    df1['ID_DIF'] = df1['ID1']-df1['ID']
    fen_list = list(df1[df1.ID_DIF>1].index)
    aa = []
    if len(fen_list)<1:
        return 0
    else:
        for i in range(len(fen_list)):
            if i == 0:
                aa.append(df1[:fen_list[i]+1])
            elif i == len(fen_list):
                aa.append(df1[fen_list[i]+1:])
            else:
                aa.append(df1[fen_list[i-1]+1:fen_list[i]+1])
        a = 0
        b = 0
        for df in aa:
            for i in ['0','1','2','3','4','5','6','7','8','9']:
                if i in str(list(df.ACTION_VALUE)[0]):
                    a += 1
                if i in str(list(df.ACTION_VALUE)[-1]):
                    b += 1
        c = 0
        if (a > 0)& (b == 0):
            c = 1
        return c
    
is_addr_fuzzy_df = client_addr_df1.groupby('USER_ID').apply(stat_addr)
is_addr_fuzzy_df = is_addr_fuzzy_df.reset_index().rename(columns={0:'is_addr_fuzzy'})

D:\Anaconda\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Wall time: 2min 11s


In [216]:
#################是否使用默认输入法######################

In [522]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
ll = []
userlist = []
for USER_ID in set(list(client_addr_df.USER_ID)):
    ll.append(''.join([str(addr) for addr in list(client_addr_df[client_addr_df.USER_ID==USER_ID].ACTION_VALUE)]))
    userlist.append(USER_ID)
    
is_default_input_list = []
for addrlist in ll:
    try :
        re.search(u"[a-z]+？+[a-z]",addrlist).group()
#         print(addrlist)
        is_default_input_list.append(1)
    except:
        is_default_input_list.append(0)
is_default_input_df = pd.DataFrame({'USER_ID':userlist,'is_default_input':is_default_input_list})           

Wall time: 39.7 s


In [ ]:
##############是否是手写输入#########################
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
J,一,、,亠,乛

In [ ]:
#############是否含有标点###################################

In [523]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
ll = []
userlist = []
for USER_ID in set(list(client_addr_df.USER_ID)):
    ll.append(''.join([str(addr) for addr in list(client_addr_df[client_addr_df.USER_ID==USER_ID].ACTION_VALUE)]))
    userlist.append(USER_ID)
is_obtain_someting_list = []
for l in ll:
    c = 0
    for a in ['(',')','（','）','【','】','[',']','{','}','-','_']:
        if a in l:
            c += 1
    if c == 0:
        is_obtain_someting_list.append(0)
    else:
        is_obtain_someting_list.append(1)
is_addr_include_punctuation = pd.DataFrame({'USER_ID':userlist,'is_default_input':is_obtain_someting_list})        
        

Wall time: 39 s


In [524]:
###################地址长度最大值#########################################

In [525]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
client_addr_df['addr_str_len'] = client_addr_df.ACTION_VALUE.str.len()
max_addr_len_df = client_addr_df[['USER_ID','addr_str_len']].groupby('USER_ID').max().reset_index()

Wall time: 806 ms


D:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [221]:
#################地址最终回落长度######

In [526]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
last_addr_len = client_addr_df.sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='last')[['USER_ID','ACTION_VALUE']]
last_addr_len['last_addr_len'] = last_addr_len.ACTION_VALUE.str.len()
final_fall_addr_len = pd.merge(last_addr_len,max_addr_len_df,on='USER_ID',how='inner')
final_fall_addr_len['final_fall_addr_len'] = final_fall_addr_len.addr_str_len-final_fall_addr_len.last_addr_len
final_fall_addr_len_df = final_fall_addr_len[['USER_ID','final_fall_addr_len']]

Wall time: 1.27 s


In [ ]:
####################地址最大回落长度################################

In [527]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
client_addr_df['addr_str_len'] = client_addr_df.ACTION_VALUE.str.len()
client_addr_df['addr_str_len_b'] = client_addr_df['addr_str_len'].shift(1)
client_addr_df['addr_str_len_a'] = client_addr_df['addr_str_len'].shift(-1)
def get_minmum(df):
    aa = []
    if len(df)>=3:
        for i in range(1,len(list(df.addr_str_len))-1):
            if (list(df.addr_str_len)[i]<list(df.addr_str_len)[i-1])&(list(df.addr_str_len)[i]<list(df.addr_str_len)[i+1]):
                aa.append(list(df.addr_str_len)[i])
            else:
                aa.append(9999)
    else:
        aa.append(9999)
    return min(aa)
max_fall_addr_len = client_addr_df.groupby('USER_ID').apply(get_minmum)
max_fall_addr_len = pd.merge(max_addr_len_df,max_fall_addr_len.reset_index(),on='USER_ID',how='inner')
max_fall_addr_len['max_fall_addr_len'] = max_fall_addr_len.addr_str_len-max_fall_addr_len[0]
max_fall_addr_len.ix[max_fall_addr_len.max_fall_addr_len<0,'max_fall_addr_len'] = -1
max_fall_addr_len_df = max_fall_addr_len[['USER_ID','max_fall_addr_len']]

D:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
D:\Anaconda\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
D:\Anaconda\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

Wall time: 49.8 s


In [225]:
#############城区乡镇####################

In [528]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
client_addr_df = client_addr_df.sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='last')[['USER_ID','ACTION_VALUE']]
addr_list = list(client_addr_df.ACTION_VALUE)
addr_list_box = []
for i in range(len(addr_list)):
    if ('镇' in addr_list[i]) |('乡' in addr_list[i]):
        addr_list_box.append(1)
    else:
        addr_list_box.append(0)
client_addr_df['city_township'] = addr_list_box
city_township_df = client_addr_df[['USER_ID','city_township']]

Wall time: 1.31 s


In [ ]:
##########是否有过复制粘贴#########################

In [529]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
client_addr_df = client_addr_df.sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='first')[['USER_ID','ACTION_VALUE']]
client_addr_df['addr_str_len'] = client_addr_df.ACTION_VALUE.str.len()
client_addr_df['is_addr_copypaste'] = 0
client_addr_df.ix[client_addr_df['addr_str_len']>=5,'is_addr_copypaste'] = 1
is_addr_copypaste_df = client_addr_df[['USER_ID','is_addr_copypaste']]

Wall time: 1.25 s


In [ ]:
##############详细地址中是否包含市县区###################################

In [530]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
client_addr_df = client_addr_df.sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='last')[['USER_ID','ACTION_VALUE']]
addr_list = list(client_addr_df.ACTION_VALUE)
addr_list_box = []
for i in range(len(addr_list)):
    if ('市' in addr_list[i]) |('区' in addr_list[i]) |('县' in addr_list[i]):
        addr_list_box.append(1)
    else:
        addr_list_box.append(0)
client_addr_df['is_addr_includecity'] = addr_list_box
is_addr_includecity_df = client_addr_df[['USER_ID','is_addr_includecity']]

Wall time: 1.25 s


In [ ]:
###############输入地址熟练度############################################

In [531]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
client_addr_df = client_addr_df.sort_values(by=['USER_ID','ID'])
client_addr_df['addr_str_len'] = client_addr_df.ACTION_VALUE.str.len()
def cal_pro(df):
    df['addr_str_len1'] = df['addr_str_len'].shift(-1).fillna(0)
    df['dif'] = df['addr_str_len1']-df['addr_str_len']
    df1 = df[df.dif>0]
    if len(df1)>0:
        a = sum(df1.dif)/len(df1)
    else:
        a = -1
    return a
input_addr_proficiency_df = client_addr_df.groupby('USER_ID').apply(cal_pro)


Wall time: 56.4 s


In [244]:
####################总编辑次数####################

In [532]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
addr_id_cnt_df = client_addr_df[['USER_ID','ID']].groupby('USER_ID').count().reset_index().rename(columns={'ID':'addr_id_cnt'})

Wall time: 410 ms


In [254]:
###################首次输入字符长度#########################

In [255]:
%%time
client_addr_df = client_info_df[client_info_df.ACTION_CONTROL=='my_home_address']
client_addr_df = client_addr_df.sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='first')[['USER_ID','ACTION_VALUE']]
client_addr_df['addr_str_len'] = client_addr_df.ACTION_VALUE.str.len()
first_addrinput_strlen_df = client_addr_df[['USER_ID','addr_str_len']].rename(columns={'addr_str_len':'first_addrinput_strlen'})